In [8]:
import re
import sys
import os
import datetime

from __future__ import print_function

In [10]:
from pyspark.sql.functions import col, sum
from pyspark.sql.functions import lit, concat
from pyspark.sql import functions as sqlFunctions
from pyspark.sql.functions import desc
from pyspark.sql.functions import dayofmonth

#from spark_notebook_helpers import prepareSubplot, np, plt, cm

In [11]:
# пример использования RE
# m = re.search('(?<=abc)def', 'abcdef')
# m.group(0)

## EDA

Используем данные логов от NASA Kennedy Space Center и проведем анализ

### Load

Создадим `sqlContext` и прочитаем данные `sqlContext.read.text()`

In [22]:
base_df = sqlContext.read.text(file_path)
base_df.printSchema()

In [24]:
base_df.show(truncate=False)

### Парсинг данных

Файлы представлены в формате [Common Log Format](https://www.w3.org/Daemon/User/Config/Logging.html#common-logfile-format)

_remotehost rfc931 authuser [date] "request" status bytes_

| field         | meaning                                                                |
| ------------- | ---------------------------------------------------------------------- |
| _remotehost_  | Remote hostname (or IP number if DNS hostname is not available).       |
| _rfc931_      | The remote logname of the user. We don't really care about this field. |
| _authuser_    | The username of the remote user, as authenticated by the HTTP server.  |
| _[date]_      | The date and time of the request.                                      |
| _"request"_   | The request, exactly as it came from the browser or client.            |
| _status_      | The HTTP status code the server sent back to the client.               |
| _bytes_       | The number of bytes (`Content-Length`) transferred to the client.      |


Используем [regexp\_extract()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.regexp_extract) для извлечения данных.

Если вы хотите познакомиться с Regular Expressions ближе, то рекомендую книгу [_Regular Expressions Cookbook_](http://shop.oreilly.com/product/0636920023630.do)

In [27]:
from pyspark.sql.functions import split, regexp_extract
split_df = base_df.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('host'),
                          regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('timestamp'),
                          regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('path'),
                          regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('status'),
                          regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('content_size'))
split_df.show(truncate=False)

### Почистим данные

In [29]:
# счет пропусков
base_df.filter(base_df['value'].isNull()).count()

Если парсинг правильный, то пустых не должно быть

In [31]:
bad_rows_df = split_df.filter(split_df['host'].isNull() |
                              split_df['timestamp'].isNull() |
                              split_df['path'].isNull() |
                              split_df['status'].isNull() |
                             split_df['content_size'].isNull())
bad_rows_df.count()

In [33]:
# подсчет по каждой колонке
def count_null(col_name):
    return sum(col(col_name).isNull().cast('integer')).alias(col_name)


exprs = []
for col_name in split_df.columns:
    exprs.append(count_null(col_name))

# сделаем агрегат по содержанию в exprs
split_df.agg(*exprs).show()

In [35]:
# кол-во не подходящего контента
bad_content_size_df = base_df.filter(~ base_df['value'].rlike(r'\d+$'))
bad_content_size_df.count()

In [37]:
# заглянем в нутрь плохих строк
bad_content_size_df.select(concat(bad_content_size_df['value'], lit('*'))).show(truncate=False)

### Исправим плохие строки

Воспользуемся 
* [fillna()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.fillna)
* [na](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.na) возвращает [DataFrameNaFunctions](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameNaFunctions) 


In [40]:
# заменим все content_size на 0.
cleaned_df = split_df.na.fill({'content_size': 0})

In [41]:
# проверим, что пустот больше нет
exprs = []
for col_name in cleaned_df.columns:
    exprs.append(count_null(col_name))

cleaned_df.agg(*exprs).show()

### Парсинг TimeStamp

Создадим UDF для парсинга

А какие способы обработки временных данных вы знаете? (для моделей машинного обучения?)

Вот вам примеры:

- [stackexchange](https://stats.stackexchange.com/questions/126230/optimal-construction-of-day-feature-in-neural-networks)
- [kaggle](https://www.kaggle.com/avanwyk/encoding-cyclical-features-for-deep-learning)
- [stackexchange another](https://datascience.stackexchange.com/questions/5990/what-is-a-good-way-to-transform-cyclic-ordinal-attributes)
- [github](https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/)
- [medium](https://medium.com/ai%C2%B3-theory-practice-business/top-6-errors-novice-machine-learning-engineers-make-e82273d394db)
- [towardsdatascience](https://towardsdatascience.com/cyclical-features-encoding-its-about-time-ce23581845ca)


In [ ]:
month_map = {
  'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
  'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12
}

def parse_clf_time(s):
    """ 
    Args:
        s (str): date and time in Apache time format [dd/mmm/yyyy:hh:mm:ss (+/-)zzzz]
    Returns:
        a string  to CAST('timestamp')
    """
    return "{0:04d}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}".format(
      int(s[7:11]),
      month_map[s[3:6]],
      int(s[0:2]),
      int(s[12:14]),
      int(s[15:17]),
      int(s[18:20])
    )

u_parse_time = udf(parse_clf_time)

logs_df = cleaned_df.select('*', u_parse_time(cleaned_df['timestamp']).cast('timestamp').alias('time')).drop('timestamp')
total_log_entries = logs_df.count()

In [44]:
logs_df.printSchema()

In [45]:
display(logs_df)

In [47]:
logs_df.cache()

##  Базовый анализ

`.describe()` для получения: count, mean, stddev, min и max

In [49]:
content_size_summary_df = logs_df.describe(['content_size'])
content_size_summary_df.show()

Или применим что-нибудь из набора функций `pyspark.sql.functions` - [documentation](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions).

In [51]:
# получите минимальное, среднее и максимальное значение по content_size
content_size_stats =  (logs_df
                       #
                       # ваш код здесь
                       #
                       .first())

print('Using SQL functions:')
print('Content Size Avg: {1:,.2f}; Min: {0:.2f}; Max: {2:,.0f}'.format(*content_size_stats))

### HTTP Status 

Сделаем обзор данных по `status` и применим сортировку для сгруппированного объекта по типу `status`
! Не забудем сделать `cache`

In [ ]:
# получите кол-во элементов по каждому status и отсортируйте по status
status_to_count_df =(logs_df
                        #
                        # ваш код здесь
                        #
                    )
                     
                     
                     
status_to_count_length = status_to_count_df.count()
print('Found %d response codes' % status_to_count_length)
status_to_count_df.show()

### Визуализация

Для визуализации используйте `display()` 

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/plot_options_1.png" style="float: right; margin-right: 30px; border: 1px solid #999999"/>

In [55]:
display(status_to_count_df)

In [57]:
# из-за большой разницы в количестве, нет хорошего отображения данных
# что можно применить, чтобы исправить визализацию?
log_status_to_count_df = status_to_count_df.withColumn('log(count)', sqlFunctions.log(status_to_count_df['count']))

display(log_status_to_count_df)

Добавим ещё одну библиотеку в нашу работу [`spark_notebook_helpers`](https://pypi.python.org/pypi/spark_notebook_helpers/1.0.1), она помогает визуализировать и узнавать объекты Spark

![](notebook_helper.jpg)

In [60]:
# пример
help(prepareSubplot)

Для визуализации используем matplotlib и набор "Set1". Другие наборы на [сайте](http://matplotlib.org/examples/color/colormaps_reference.html). Разные схемы цветов позволяют сделать разные акценты

In [62]:
data = log_status_to_count_df.drop('count').collect()
# x, y = zip(*data)
# index = np.arange(len(x))
# bar_width = 0.7
# colorMap = 'Set1'
# cmap = cm.get_cmap(colorMap)

# fig, ax = prepareSubplot(np.arange(0, 6, 1), np.arange(0, 14, 2))
# plt.bar(index, y, width=bar_width, color=cmap(0))
# plt.xticks(index + bar_width/2.0, x)
# display(fig)

### Определить частоту встречамости Host

Создадим новый DF, который будет включать агрегаты по `host` и их количество.
Сделаем фильтр по количеству, чтобы в DF остались хосты, которые встречаются больше 10

In [64]:
host_sum_df =(logs_df
              .groupBy('host')
              .count())

host_more_than_10_df = (host_sum_df
                        .filter(host_sum_df['count'] > 10)
                        .select(host_sum_df['host']))

print('Any 20 hosts that have accessed more then 10 times:\n')
host_more_than_10_df.show(truncate=False)

### Визуализация Path

Сделаем группированный по `path` DF, который будет сортированный по убыванию количества встречания `path` 

Для визуазиции данных, мы извлечем `path` и `count` из DF в объект PairRDD (к каждой `Rows` применить функцию `lambda` через `map`)

In [66]:
paths_df = (logs_df
            .groupBy('path')
            .count()
            .sort('count', ascending=False)
           )

paths_counts = (paths_df
                .select('path', 'count')
                .map(lambda r: (r[0], r[1]))
                .collect())

paths, counts = zip(*paths_counts)

# colorMap = 'Accent'
# cmap = cm.get_cmap(colorMap)
# index = np.arange(1000)

# fig, ax = prepareSubplot(np.arange(0, 1000, 100), np.arange(0, 70000, 10000))
# plt.xlabel('Paths')
# plt.ylabel('Number of Hits')
# plt.plot(index, counts[:1000], color=cmap(0), linewidth=3)
# plt.axhline(linewidth=2, color='#999999')
# display(fig)

In [68]:
display(paths_df)

### Top Paths

Используя метод `.show()` извлечем ТОП `n=10` и отобразим `truncate=False`

In [ ]:
# Top Paths
print('Top Ten Paths:')
paths_df.show(n=10, truncate=False)


### Аналитика по Log File


**Top Ten Error Paths**

Код нормальной работы в web - `200`. А сколько было сделано запросов с другими кодами?

Сделайте DF, который покажет, сколько было запросов, которые вернули нет 200 код

In [73]:
# Фильтр по DataFrame, чтобы убрать все 200
not200DF = logs_df.filter(# ваш код здесь)
not200DF.show(10)

# Сделаем группировку по path и сортировку по убыванию (по количеству)
logs_sum_df = (not200DF
               .groupby('path')
               .count()
               .sort(desc('count'))
               .cache())

print()'Top Ten failed URLs:')
logs_sum_df.show(10, False)

In [74]:
# сделаем PairRDD по Топ 10 ошибок
top_10_err_urls = [(row[0], row[1]) for row in logs_sum_df.take(10)]

### Количество уникальных Host


In [76]:
# найдем уникальные Host и посчитаем их количество
unique_host_count = (logs_df
                     #
                     # ваш код здесь
                     #
                    ).count()


print('Unique hosts: {0}'.format(unique_host_count))

### Количество уникальных  Host в день

Используя функцию [`dayofmonth` function](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.dayofmonth) из `pyspark.sql.functions` модуля, найдите уникальное количество host на каждый день.


**`day_to_host_pair_df`**

DataFrame должен содержать следующие колонки

| column | explanation          |
| ------ | -------------------- |
| `host` | имя host             |
| `day`  | день в месяце        |


В каждой строке `logs_df` содержится дата и время:

```
gw1.att.com - - [23/Aug/1995:00:03:53 -0400] "GET /shuttle/missions/sts-73/news HTTP/1.0" 302 -
```

Из предыдущей строки можно извлечь для `day_to_host_pair_df`:

```
gw1.att.com 23

[23/Aug/1995:00:03:53 -0400]
```

**`day_group_hosts_df`**

DataFrame `day_to_host_pair_df` содержит сгруппированные данные по (`day`, `host`), без дубликатов.

**`daily_hosts_df`**

Результатом должен быть DataFrame

| column  | explanation                                        |
| ------- | -------------------------------------------------- |
| `day`   | день месяца                                        |
| `count` | кол-во уникальных хостов                           |

In [79]:
day_to_host_pair_df = logs_df.select(logs_df.host,
                                     dayofmonth('time').alias('day'))

day_group_hosts_df = day_to_host_pair_df.distinct()

daily_hosts_df = day_group_hosts_df.select(day_group_hosts_df['day']).groupby(day_group_hosts_df['day']).count().cache()

print('Unique hosts per day: \n',daily_hosts_df.show(10))
daily_hosts_df.show(30, False)

### Визуализация уникальных

**WARNING**: Запомним, что `collect()` возвращает список из `Row`, данные надо будет обработать и после `collect()`

In [82]:
# список дней
days_with_hosts = daily_hosts_df.select(daily_hosts_df['day'])

# списко уникальных хостов
hosts = daily_hosts_df.select(daily_hosts_df['count'])

days_with_hosts, hosts = [list(i) for i in zip(*daily_hosts_df.select('day', 'count').map(lambda r: (r[0], r[1])).collect())]
print(days_with_hosts)
print(hosts)

In [84]:
# fig, ax = prepareSubplot(np.arange(0, 30, 5), np.arange(0, 5000, 1000))
# colorMap = 'Dark2'
# cmap = cm.get_cmap(colorMap)
# plt.plot(days_with_hosts, hosts, color=cmap(0), linewidth=3)
# plt.axis([0, max(days_with_hosts), 0, max(hosts)+500])
# plt.xlabel('Day')
# plt.ylabel('Hosts')
# plt.axhline(linewidth=3, color='#999999')
# plt.axvline(linewidth=2, color='#999999')
# display(fig)

In [86]:
display(daily_hosts_df)

### Среднее по запросу в день на каждый Host


In [88]:
# расчет среднего по хосту в день = (все запросы в день)/(количество уникальных хостов в день)

# всего запросов в день
total_req_per_day_df = logs_df.select(dayofmonth('time').alias('day')).groupby('day').count()

print(total_req_per_day_df.show(10))



# расчет среднего
avg_daily_req_per_host_df = (
  total_req_per_day_df.join( daily_hosts_df, daily_hosts_df['day'] == total_req_per_day_df['day'])
   .drop(daily_hosts_df['day'])
   .select(
    total_req_per_day_df['day'], (total_req_per_day_df['count'] / daily_hosts_df['count']).alias('avg_reqs_per_host_per_day')
  )
).cache()

print('Average number of daily requests per Hosts is:\n')
avg_daily_req_per_host_df.show()

## Part 5: Exploring 404 Status Codes

Let's drill down and explore the error 404 status records. We've all seen those "404 Not Found" web pages. 404 errors are returned when the server cannot find the resource (page or object) the browser or client requested.

### (5a) Exercise: Counting 404 Response Codes

Create a DataFrame containing only log records with a 404 status code. Make sure you `cache()` `not_found_df` as we will use it in the rest of this exercise.

How many 404 records are in the log?

In [98]:
# TODO: Replace <FILL IN> with appropriate code

not_found_df = logs_df.filter(logs_df['status'] == '404').cache()
print('Found {0} 404 URLs').format(not_found_df.count()) 

In [99]:
# TEST Counting 404 (5a)
Test.assertEquals(not_found_df.count(), 6185, 'incorrect not_found_df.count()')
Test.assertTrue(not_found_df.is_cached, 'incorrect not_found_df.is_cached')

### Исследование 404

Сделаем исследование ошибки 404 в логах

In [101]:
# выбор только 404
not_found_paths_df = not_found_df.select('path').cache()
# только уникальные пути с ошибкой 404
unique_not_found_paths_df = not_found_paths_df.distinct()

print('404 URLS:\n')
unique_not_found_paths_df.show(n=40, truncate=False)

In [104]:
# top 20 путей по ошибке 404
top_20_not_found_df = not_found_paths_df.groupby('path').count().sort(desc('count')).cache()

print('Top Twenty 404 URLs:\n')
top_20_not_found_df.show(n=20, truncate=False)

### Решите самостоятельно

In [107]:
# топ 25 Host, которые создают ошибку

In [119]:
# топ 5 дней с ошибками 404

In [122]:
# ошибки 404 по часам